In [ ]:
"""
Análise Exploratória dos Top 250 Filmes do IMDb

Objetivos:
- Analisar distribuição de notas
- Identificar gêneros mais frequentes
- Explorar relações entre variáveis

Autor: [Lets]
Data: [16/08/2025]
Versão: 1.0
"""

# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import logging

# Configuração de logging
logging.basicConfig(
    filename='analysis.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Configurações de estilo
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
pd.set_option('display.max_columns', None)

def load_data(filepath):
    """Carrega os dados do arquivo CSV"""
    try:
        df = pd.read_csv(filepath)
        logging.info("Dataset carregado com sucesso")
        print("✅ Dataset carregado com sucesso!")
        return df
    except FileNotFoundError as e:
        logging.error(f"Arquivo não encontrado: {e}")
        print(f"❌ Erro: Arquivo não encontrado em {filepath}")
        exit()
    except Exception as e:
        logging.error(f"Erro inesperado: {e}")
        print(f"❌ Erro inesperado: {e}")
        exit()


# Carregamento dos dados
df = load_data("data/imdb_top250.csv")

# Inspeção inicial
print("\n🔍 Primeiras linhas:")
print(df.head(3))

print("\n📊 Informações do dataset:")
print(df.info())

print("\n📈 Estatísticas descritivas:")
print(df.describe())

# Tratamento de dados
df['Duration'] = df['Duration'].str.replace(' min', '').astype(int)
df['Votes'] = df['Votes'].str.replace(',', '').astype(int)
df['Main_Genre'] = df['Genre'].str.split(',').str[0].str.strip()
df['Decade'] = (df['Year'] // 10) * 10

# Validação de dados
assert df['Rating'].between(0, 10).all(), "Valores de Rating fora do intervalo 0-10"

# Verificação de qualidade
print("\n🧹 Valores nulos por coluna:")
print(df.isnull().sum())

print("\n♻️ Linhas duplicadas:", df.duplicated().sum())

def plot_histogram(data, column, title, xlabel, color="#3498db", bins=20):
    """Cria histograma com linha de média"""
    plt.figure(figsize=(12, 6))
    sns.histplot(data[column], bins=bins, kde=True, color=color)
    plt.title(title, fontsize=16, pad=20)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel('Frequência', fontsize=12)
    plt.axvline(data[column].mean(), color='#e74c3c', linestyle='--',
                label=f'Média: {data[column].mean():.2f}')
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_top_categories(data, column, title, top_n=10):
    """Plota as top categorias de uma coluna"""
    counts = data[column].value_counts().head(top_n)
    plt.figure(figsize=(14, 7))
    sns.barplot(x=counts.values, y=counts.index, palette='viridis')
    plt.title(title, fontsize=16, pad=20)
    plt.xlabel('Contagem', fontsize=12)
    plt.ylabel(column, fontsize=12)
    plt.tight_layout()
    plt.show()

# Visualizações estáticas (Seaborn / Matplotlib)
plot_histogram(df, 'Rating', 'Distribuição das Notas no IMDb Top 250', 'Nota (0-10)')
plot_top_categories(df, 'Main_Genre', 'Top 10 Gêneros no IMDb Top 250')

# Visualizações interativas (Plotly)
fig = px.histogram(df, x="Rating", nbins=20, 
                   title="Distribuição das Notas IMDb",
                   color_discrete_sequence=["#3498db"])
fig.update_layout(bargap=0.1)
fig.show()


